In [1]:

import glob
import tqdm
import torch
import json
import os
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import tensorboard


# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
import time
import copy

from model import *
from dataset import *

In [43]:

def collate_fn(wqi, data_size):
    wqi = wqi
    len_wqi = len(wqi)
    len_0 = data_size - len_wqi
    
    if len_wqi != data_size:
        padd = [0 for _ in range(len_0)]
        result = padd + wqi
    
    else:
        result = wqi

    return result

class KwenDataset(Dataset):
    
    def __init__(self, path, train=True, transform=None, lstm = None):
        self.path = path
        self.img_list = os.listdir(os.path.join(path,'img'))
        self.transform = transform
        self.lstm = lstm
        with open(os.path.join(path, 'label.json'), 'r') as f:
            self.label = json.load(f)
        
        with open(os.path.join(path, 'wqi_score_sorted.json'), 'r') as f:
            self.wqi_score = json.load(f)
        
    def __len__(self):
        return len(self.img_list)
    
    

    
    def __getitem__(self, idx):
        # ex:  33.24675_126.571777_500_070925.jpg
        file_name = self.img_list[idx]
        img_path = os.path.join(self.path, 'img',file_name)
        
        label = self.label[file_name]
        
        img = Image.open(img_path)
        img = img.resize((256,192))        
        if self.transform is not None:
            img = self.transform(img)

        if self.lstm is not None:
            lat_loc = self.img_list[idx].split('_')[0]+'_'+self.img_list[idx].split('_')[1]
            
            wqi_vals = list(self.wqi_score[lat_loc].values())
            wqi_key = wqi_vals.index(label)
            wqi_pre = wqi_vals[:wqi_key+1]

            if len(wqi_pre) >=30:
                wqi_pre = wqi_pre[-30:]
            wqi = collate_fn(wqi_pre, 30)

            
            return  wqi, label
        else:
            return img, label

In [2]:
transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform, lstm = True)
dataloader =DataLoader(dataset=dataset,batch_size=32,shuffle=True,drop_last=False)

In [3]:
class RNNNet(nn.Module):
    def __init__(self, num_classes, init_weights=True):
        super().__init__()
        # lstm
        self.input_size = 1000
        self.hidden_size = 16
        self.bidirectional = True
        self.rnn = torch.nn.LSTM(self.input_size, self.hidden_size, dropout = 0, batch_first = True, bidirectional = self.bidirectional)
        self.linear3 = nn.Linear(16*(int(self.bidirectional)+1), num_classes)

    def forward(self, x):
        x, (hidden_state, cell_state) = self.rnn(x)
        x = self.linear3(x)
        #return torch.argmax(x, dim=1)
        return x
    


In [9]:
writer = SummaryWriter()
losss = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNNNet(1).to(device)

transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform, lstm = True)
dataloader =DataLoader(dataset=dataset,batch_size=32,shuffle=True,drop_last=False)


criterion = nn.MSELoss().to(device)
learning_rate = 10
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

decive = 'cuda'
count_idx = 0
for epoch in range(10):
    model.train()
    print(f"epoch : {epoch} ")
    for batch in tqdm.tqdm(dataloader):
        label, wqi = batch
        optimizer.zero_grad()
        x= torch.stack(wqi).to(device)
        x = torch.transpose(x,0,1)
        x = x.to(torch.float32)
        
        label = label.to(device)
        print(x)
        print(label)
        
        output = model(x)
        output = output.view(-1)
        
        label = label.to(torch.float32)
        output = output.to(torch.float32)
        
        loss = criterion(output, label)
        
        loss = loss.to(torch.float32)
        loss.backward()
        
        optimizer.step()
        count_idx += 1
        writer.add_scalar('log', loss, count_idx)
        losss[count_idx] = loss.item()
        #print("lr: ", optimizer.param_groups[0]['lr'])
        
        if count_idx%10 == 1 :
            optimizer.param_groups[0]['lr'] *= 0.995
            #print('output: \t', output)
            #print('label  :\t', label)
            print('loss:\t', loss.item())
            #print('output: \t', output)
            #print('label  :\t', label)
            #print('loss:\t', loss)
            #print('lr: ', optimizer.param_groups[0]['lr'])

torch.save(model.state_dict(), './model_state_dict.pd')

writer.flush()
writer.close()

epoch : 0 


  0%|          | 0/1066 [00:00<?, ?it/s]


UnboundLocalError: local variable 'padd' referenced before assignment

In [65]:
x.size()

torch.Size([3, 1000])

In [62]:
a = torch.transpose(x,0,1)

In [63]:
a

tensor([[ 0.0000,  0.0000,  0.0000,  ..., 88.0100, 88.7200, 91.8200],
        [ 0.0000,  0.0000,  0.0000,  ..., 90.7400, 69.1000, 75.0900],
        [ 0.0000,  0.0000,  0.0000,  ..., 80.6700, 91.5600, 87.4300]],
       device='cuda:0', dtype=torch.float64)

In [22]:
for batch in dataloader:
    print(len(batch))
    break

2


In [14]:


class SEBlock(nn.Module):
    def __init__(self, in_channels, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // r),
            nn.ReLU(),
            nn.Linear(in_channels // r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.squeeze(x)
        x = x.view(x.size(0), -1) 
        x = self.excitation(x)
        x = x.view(x.size(0), x.size(1), 1, 1)
        return x

# Depthwise Separable Convolution
class Depthwise(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.depthwise = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU6(),
        )

        self.pointwise = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6(),
        )

        self.seblock = SEBlock(out_channels)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        x = self.seblock(x) * x
        return x


# BasicConv2d
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, **kwargs):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU6()
        )

    def forward(self, x):
        x = self.conv(x)
        return x


# MobileNetV1
class MobileNet(nn.Module):
    def __init__(self, width_multiplier, num_classes, init_weights=True):
        super().__init__()
        self.init_weights=init_weights
        alpha = width_multiplier

        self.conv1 = BasicConv2d(3, int(32*alpha), 3, stride=2, padding=1)
        self.conv2 = Depthwise(int(32*alpha), int(64*alpha), stride=1)
        # down sample
        self.conv3 = nn.Sequential(
            Depthwise(int(64*alpha), int(128*alpha), stride=2),
            Depthwise(int(128*alpha), int(128*alpha), stride=1)
        )
        # down sample
        self.conv4 = nn.Sequential(
            Depthwise(int(128*alpha), int(256*alpha), stride=2),
            Depthwise(int(256*alpha), int(256*alpha), stride=1)
        )
        # down sample
        self.conv5 = nn.Sequential(
            Depthwise(int(256*alpha), int(512*alpha), stride=2),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
            Depthwise(int(512*alpha), int(512*alpha), stride=1),
        )
        # down sample
        self.conv6 = nn.Sequential(
            Depthwise(int(512*alpha), int(1024*alpha), stride=2)
        )
        # down sample
        self.conv7 = nn.Sequential(
            Depthwise(int(1024*alpha), int(1024*alpha), stride=2)
        )
        # linear
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.linear = nn.Linear(int(1024*alpha), 256)
        #self.linear2 = nn.Linear( 256, 16)


        # lstm
        self.input_size = 256
        self.hidden_size = 16
        self.bidirectional = True
        self.rnn = torch.nn.LSTM(self.input_size, self.hidden_size, dropout = 0, batch_first = True, bidirectional = self.bidirectional)
        self.linear3 = nn.Linear(16*(int(self.bidirectional)+1), num_classes)
        
        # weights initialization
        if self.init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        #x = self.linear2(x)
        x, (hidden_state, cell_state) = self.rnn(x)
        x = self.linear3(x)
        #return torch.argmax(x, dim=1)
        return x
    
    # weights initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def mobilenet(alpha=1, num_classes=1):
    return MobileNet(alpha, num_classes)
            

In [ ]:
decive = 'cuda'
writer = SummaryWriter()
losss = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = mobilenet().to(device)

transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform)
dataloader =DataLoader(dataset=dataset,batch_size=16,shuffle=True,drop_last=False)

#criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.MSELoss().to(device)
learning_rate = 10
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


count_idx = 0
for epoch in range(10):
    model.train()
    print(f"epoch : {epoch} ")
    for batch in tqdm.tqdm(dataloader):
        img, label = batch
        optimizer.zero_grad()
        x = img.to(device)
        label = label.to(device)
        output = model(x)
        output = output.view(-1)
        
        label = label.to(torch.float32)
        output = output.to(torch.float32)
        
        loss = criterion(output, label)
        
        loss = loss.to(torch.float32)
        loss.backward()
        
        optimizer.step()
        count_idx += 1
        writer.add_scalar('log', loss, count_idx)
        losss[count_idx] = loss.item()
        #print("lr: ", optimizer.param_groups[0]['lr'])
        
        if count_idx%10 == 1 :
            optimizer.param_groups[0]['lr'] *= 0.995
            #print('output: \t', output)
            #print('label  :\t', label)
            print('loss:\t', loss)
            #print('lr: ', optimizer.param_groups[0]['lr'])

        if count_idx%1000 ==1 :
            print('output: \t', output)
            print('label  :\t', label)
            print('loss:\t', loss)
            print('lr: ', optimizer.param_groups[0]['lr'])

torch.save(model.state_dict(), './model_state_dict.pd')

writer.flush()
writer.close()

In [15]:
writer = SummaryWriter()
losss = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = mobilenet().to(device)

transform = transforms.Compose([transforms.ToTensor(),])
dataset = KwenDataset(path = 'dataset', transform= transform, lstm=True)
dataloader =DataLoader(dataset=dataset,batch_size=16,shuffle=True,drop_last=False)

#criterion = nn.CrossEntropyLoss().to(device)
criterion = nn.MSELoss().to(device)
learning_rate = 10
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

decive = 'cuda'
count_idx = 0
for epoch in range(10):
    model.train()
    print(f"epoch : {epoch} ")
    for batch in tqdm.tqdm(dataloader):
        print(len(batch))
        break

epoch : 0 


  0%|          | 0/2132 [00:00<?, ?it/s]


RuntimeError: each element in list of batch should be of equal size

In [42]:
torch.cuda.empty_cache()

In [19]:
import pandas as pd
a =pd.NA

In [16]:
a =  {
		"010601": 76.76,
		"011101": 70.99000000000001,
		"020601": 71.87,
		"021001": 83.09,
		"030401": 81.17999999999999,
		"031001": 82.61,
		"040501": 88.43,
		"041001": 78.27,
		"050401": 78.21,
		"050601": 74.76,
		"050801": 71.41,
		"051101": 75.28,
		"060301": 84.36,
		"060601": 73.4,
		"060801": 78.76,
		"061101": 78.96000000000001,
		"070401": 92.01,
		"070601": 67.8,
		"070801": 74.59,
		"071101": 82.52,
		"090201": 90.32,
		"090401": 88.61,
		"090701": 84.19000000000001,
		"091001": 76.41,
		"100201": 85.38,
		"100401": 84.0,
		"100701": 81.2,
		"101001": 74.45,
		"110101": 87.15,
		"110401": 89.97,
		"110701": 89.01,
		"111001": 71.31,
		"120201": 77.36000000000001,
		"120501": 89.05,
		"120701": 76.33,
		"121001": 76.57000000000001,
		"130301": 85.82,
		"130501": 76.0,
		"130701": 77.53999999999999,
		"131001": 77.73,
		"140401": 87.86999999999999,
		"140601": 77.8,
		"140901": 74.9,
		"141101": 82.06
	}

In [17]:
a.values()

dict_values([76.76, 70.99000000000001, 71.87, 83.09, 81.17999999999999, 82.61, 88.43, 78.27, 78.21, 74.76, 71.41, 75.28, 84.36, 73.4, 78.76, 78.96000000000001, 92.01, 67.8, 74.59, 82.52, 90.32, 88.61, 84.19000000000001, 76.41, 85.38, 84.0, 81.2, 74.45, 87.15, 89.97, 89.01, 71.31, 77.36000000000001, 89.05, 76.33, 76.57000000000001, 85.82, 76.0, 77.53999999999999, 77.73, 87.86999999999999, 77.8, 74.9, 82.06])

In [21]:
criterion = nn.MSELoss().to(device)
criterion(output, label)

tensor(7317.9502, device='cuda:0', grad_fn=<MseLossBackward0>)

In [16]:
output2

tensor([-5.5138e-05, -2.1082e-04, -1.4001e-04, -2.4391e-04, -1.9421e-04,
        -2.8133e-05,  6.1817e-05, -1.5899e-04, -1.1474e-04, -1.6041e-04,
        -2.0102e-05, -1.0662e-04,  6.0250e-05,  2.9699e-05,  3.6472e-05,
         4.1496e-05], device='cuda:0', grad_fn=<ViewBackward0>)

In [14]:
output

tensor([[-5.5138e-05],
        [-2.1082e-04],
        [-1.4001e-04],
        [-2.4391e-04],
        [-1.9421e-04],
        [-2.8133e-05],
        [ 6.1817e-05],
        [-1.5899e-04],
        [-1.1474e-04],
        [-1.6041e-04],
        [-2.0102e-05],
        [-1.0662e-04],
        [ 6.0250e-05],
        [ 2.9699e-05],
        [ 3.6472e-05],
        [ 4.1496e-05]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
import json
with open('loss.json', 'w') as f:
    json.dump(losss, f, indent='\t')

In [ ]:
losss

In [7]:
torch.cuda.empty_cache()

In [50]:
a = torch.tensor([1,2,3])
b = torch.tensor([2,4,6])

In [51]:
torch.abs(a-b)

tensor([1, 2, 3])

In [23]:
a.size()

torch.Size([2, 4])

In [26]:
torch.argmax(a,dim = 1)

tensor([3, 2])

In [13]:
b = nn.Softmax(a)

Softmax(
  dim=tensor([[0, 1, 2, 3],
          [4, 5, 8, 4]])
)

In [16]:
torch.max(a)

tensor(8)

In [11]:
a

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])